In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
y = df.diagnosis

In [ ]:
x =df.drop(['Unnamed: 32','id','diagnosis'], axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(y)

In [ ]:
df.columns

In [ ]:
pair_plot_features = ['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean','concavity_mean','concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']
data_std = (x - x.mean()) / (x.std())              # standardization
plot_df = pd.concat([y,data_std[pair_plot_features]],axis=1)
plot_df.head(5)
# data_std.describe()

In [ ]:
g = sns.pairplot(plot_df, hue='diagnosis',height=1.5)
g.map_lower(sns.kdeplot, levels=4, color=".2")

In [ ]:
pair_plot2_features = ['radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se']
plot2_df = pd.concat([y,data_std[pair_plot2_features]],axis=1)
plot2_df.head(5)

In [ ]:
g = sns.pairplot(plot2_df, hue='diagnosis',height=1.5)
g.map_lower(sns.kdeplot, levels=4, color=".2")

In [ ]:
pair_plot3_features = ['radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']
plot3_df = pd.concat([y,data_std[pair_plot3_features]],axis=1)
plot3_df.head(5)

In [ ]:
g = sns.pairplot(plot2_df, hue='diagnosis',height=1.5)
g.map_lower(sns.kdeplot, levels=4, color=".2")

In [ ]:
plt.figure(figsize=(12,12))
data = pd.melt(plot_df,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
sns.violinplot(x= 'features', y= 'value',hue= 'diagnosis', data = data,split=True, inner="quart")
plt.xticks(rotation = 90)

In [ ]:
plt.figure(figsize=(12,12))
data = pd.melt(plot2_df,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
sns.violinplot(x= 'features', y= 'value',hue= 'diagnosis', data = data,split=True, inner="quart")
plt.xticks(rotation = 90)

In [ ]:
plt.figure(figsize=(12,12))
data = pd.melt(plot3_df,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
sns.violinplot(x= 'features', y= 'value',hue= 'diagnosis', data = data,split=True, inner="quart")
plt.xticks(rotation = 90)

In [ ]:
sns.set(style="dark", palette="bright")
data = pd.melt(plot_df,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(15,15))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)

plt.xticks(rotation=90)

In [ ]:
sns.set(style="dark", palette="bright")
data = pd.melt(plot2_df,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(15,15))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)

plt.xticks(rotation=90)

In [ ]:
sns.set(style="dark", palette="bright")
data = pd.melt(plot3_df,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(15,15))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)

plt.xticks(rotation=90)

In [ ]:
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(data_std.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

**Feature selection with correlation and random forest classification**

In [ ]:
drop_list = ['perimeter_mean','radius_mean','compactness_mean','concave points_mean','radius_se','perimeter_se','radius_worst','perimeter_worst','compactness_worst','concave points_worst','compactness_se','concave points_se','texture_worst','area_worst','concavity_worst','fractal_dimension_mean','area_se','smoothness_worst','fractal_dimension_worst','concavity_se','symmetry_worst','concavity_mean']
x_1 = data_std.drop(drop_list,axis = 1 )        # do not modify x, we will use it later 
x_1.head()

In [ ]:
f,ax = plt.subplots(figsize=(14, 14))
sns.heatmap(x_1.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.metrics import accuracy_score

# split data train 70 % and test 30 %
x_train, x_test, y_train, y_test = train_test_split(x_1, y, test_size=0.3, random_state=42)

#random forest classifier with n_estimators=10 (default)
clf_rf = RandomForestClassifier(random_state=43)      
clr_rf = clf_rf.fit(x_train,y_train)

ac = accuracy_score(y_test,clf_rf.predict(x_test))
print('Accuracy is: ',ac)
cm = confusion_matrix(y_test,clf_rf.predict(x_test))
sns.heatmap(cm,annot=True,fmt="d")

**Recursive feature elimination (RFE) with random forest**

In [ ]:
from sklearn.feature_selection import RFE
# Create the RFE object and rank each pixel
clf_rf_3 = RandomForestClassifier()      
rfe = RFE(estimator=clf_rf_3, n_features_to_select=5, step=1)
rfe = rfe.fit(x_train, y_train)

In [ ]:
print('Chosen best 5 feature by rfe:',x_train.columns[rfe.support_])

In [ ]:
from sklearn.feature_selection import RFECV

# The "accuracy" scoring is proportional to the number of correct classifications
clf_rf_4 = RandomForestClassifier() 
rfecv = RFECV(estimator=clf_rf_4, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(x_train, y_train)

print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train.columns[rfecv.support_])